In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import pickle


In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
def getRGB(img):

    r=img[0:1024]
    g=img[1024:2048]
    b=img[2048:3072]

    rs=np.array(r).reshape(32,32)
    gs=np.array(g).reshape(32,32)
    bs=np.array(b).reshape(32,32)

    cimg=cv.merge([rs,gs,bs])
    return cimg

In [ ]:
category=["plane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [ ]:
FILE_PATH=('C:/Mic/projects/ML/dataset/cifar-10-batches-py/data_batch_1')
image_dict=unpickle(FILE_PATH)

images=np.array(image_dict[b'data']).reshape(10000,3072)
labels=np.array(image_dict[b'labels'])
print(images.shape)
print(labels.shape)

rgb_images=[]
for im in images:
    rgb_images.append(getRGB(im))
rgb_images=np.array(rgb_images)
print(rgb_images.shape)

In [ ]:
train_X=tf.cast(rgb_images/255,tf.float32)
train_Y=tf.convert_to_tensor(labels,tf.int16)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten()) # 3维 转为 1维
model.add(layers.Dense(32, activation='relu'))  
model.add(layers.Dense(10, activation='softmax')) 
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
logs = "./logs/myYolo"
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 update_freq = 10000,
                                                 profile_batch = 2)

history = model.fit(train_X, train_Y, epochs=10, callbacks=[tboard_callback])

In [ ]:
plt.imshow(rgb_images[np.random.randint(10000)])

In [ ]:
FILE_PATH=('C:/Mic/projects/ML/dataset/cifar-10-batches-py/data_batch_2')
image_dict_y=unpickle(FILE_PATH)

#print(image_dict_y)

images_y=np.array(image_dict[b'data']).reshape(10000,3072)
labels_y=np.array(image_dict[b'labels'])

rgb_images_y=[]
for im in images_y:
    rgb_images_y.append(getRGB(im))
rgb_images_y=np.array(rgb_images_y)
print(rgb_images_y.shape)




In [ ]:
test_X=tf.cast(rgb_images_y/255,tf.float32)

test_Y=model.predict(test_X)

In [ ]:
n=np.random.randint(10000)
plt.imshow(rgb_images_y[n])

cl=labels_y[n] 
print("label is: %s" %category[cl])
c=np.argmax(test_Y[n])
print("predict is: %s" %category[c])


In [ ]:
model.save("./model/myYolo")

In [ ]:
#model.summary()

outputs=model.get_layer('conv2d').output


visualization_model = tf.keras.models.Model(inputs = model.input, outputs = outputs)
train_X_c = np.array(train_X).reshape((1,) + train_X.shape)
print(train_X_c.shape)

successive_feature_maps = visualization_model.predict(train_X_c[0:2])

layer_names = [layer.name for layer in model.layers]


for layer_name, feature_map in zip(layer_names, successive_feature_maps): 
        print(feature_map)
        print(feature_map.shape)
        n_features = feature_map.shape[-1]
        size = feature_map.shape[1]
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
            x = feature_map[0, :, :, i]
            x -= x.mean()
            x /= x.std()
            x *= 64
            x += 128
            x = np.clip(x, 0, 255).astype('uint8')
            display_grid[:, i * size : (i + 1) * size] = x
            scale = 20. / n_features
            plt.figure(figsize=(scale * n_features, scale))
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')

            
